##**Кейс по NLP для смены по машинному обучению от Тинькофф в Университете "Сириус".**


###**В рамках задания необходимо обучить свою диалоговую языковую модель и сделать телеграм бота для взаимодействия с ней.**

---







## Install libraries

---



In [1]:
!pip install demoji
!pip install transformers
!pip install accelerate -U
!pip install bitsandbytes
!pip install evaluate

## Imports

---



In [2]:
import pandas as pd
import demoji
from tqdm import tqdm
tqdm.pandas()
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead, TrainingArguments, Trainer, GPT2TokenizerFast, DataCollatorForLanguageModeling, GPTQConfig
from typing import Tuple, List
from torch.utils.data import Dataset, DataLoader
import glob
from evaluate import load
import os
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Данные взяты из русскоязычного сообщества  [Docker'а](https://t.me/docker_ru)**

> #### Предобработка данных была произведена с помощью скрипта `prepare_messages.py`





In [4]:
data = pd.read_csv('/content/drive/MyDrive/sirius_data/docker_messages.csv')
data.head()

,context_3,context_2,context_1,response
0,NaN,NaN,NaN,"Молодец, ты долистал до конца :D\nЭто сообщени..."
1,NaN,NaN,"Молодец, ты долистал до конца :D\nЭто сообщени...",=)\nТак сотворяется история
2,NaN,"Молодец, ты долистал до конца :D\nЭто сообщени...",=)\nТак сотворяется история,Историю пишут победители.
3,"Молодец, ты долистал до конца :D\nЭто сообщени...",=)\nТак сотворяется история,Историю пишут победители.,Всем привет!
4,=)\nТак сотворяется история,Историю пишут победители.,Всем привет!,"🖐\nЗасунуть прод в один контейнер, норм идейка."


# Preprocessing

---




In [5]:
def preprocessing(data: pd.DataFrame) -> Tuple[pd.DataFrame]:

  """Предобработка данных, формирование input'ов и target'ов"""

  print("Input shape: ", data.shape)

  demoji.download_codes()
  data.dropna(inplace=True)
  data = data.apply(lambda x: x.str.replace("\n", " "))
  data = data.progress_apply(lambda x: x.apply(lambda y: demoji.replace(y, " ")))
  data = data[data.apply(lambda x: x.isin(["", " "]) == False)]
  data.dropna(inplace=True)
  input_data = data[data.columns[:-1]].progress_apply(lambda x: " @@ПЕРВЫЙ@@ " + x if x.name.endswith(("3", "1")) else " @@ВТОРОЙ@@ " + x)
  target_data = data[data.columns[-1]]

  print("Output shape: ", data.shape)

  return input_data, target_data

# Preparing Dataset for train

---



In [6]:
def tokenize_func(tokenizer: GPT2TokenizerFast, inp_text, max_inp_length: int) -> GPT2TokenizerFast:
  return tokenizer(
      inp_text,
      max_length=max_inp_length,
      padding="max_length",
      truncation=True,
      return_tensors='pt'
  )

In [7]:
class ConvertDataset(Dataset):
  def __init__(self, tokenize_func, tokenizer: GPT2TokenizerFast, inp_df: pd.DataFrame, target_df: pd.DataFrame, max_inp_length: int):
     self.ind_input = []
     self.ind_target = []
     for _, row in inp_df.iterrows():
      self.ind_input.append(" ".join(row))
     for row in target_df:
      self.ind_target.append(row + " " + tokenizer.eos_token)

     self.ind_input = tokenize_func(tokenizer, self.ind_input, max_inp_length)
     self.ind_target = tokenize_func(tokenizer, self.ind_target, max_inp_length)

  def __len__(self):
    return len(self.ind_input['input_ids'])

  def __getitem__(self, index):
    return {
            "input_ids": self.ind_input['input_ids'][index],
            "target_ids": self.ind_target['input_ids'][index]
           }


In [8]:
tokenizer = AutoTokenizer.from_pretrained('tinkoff-ai/ruDialoGPT-medium')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
batch_size = 6
max_length = 64
input_df, target_df = preprocessing(data)

<ipython-input-5-71cebec1a521>:7: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


Input shape:  (307852, 4)


100%|██████████| 3/3 [00:00<00:00, 49.92it/s]

Output shape:  (57729, 4)


In [10]:
x_train, x_eval, y_train, y_eval = train_test_split(input_df, target_df, test_size=0.1, shuffle=True)

In [11]:
train_dataset = ConvertDataset(tokenize_func, tokenizer, x_train, y_train, max_length)

eval_dataset = ConvertDataset(tokenize_func, tokenizer, x_eval, y_eval, max_length)


# Training process

---



In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [45]:
class TrainPipeline:
  def __init__(self, tokenizer: GPT2TokenizerFast, train_dataset: pd.DataFrame, eval_dataset: pd.DataFrame, result_dir: str, model_url="tinkoff-ai/ruDialoGPT-medium"):
    self.model_url = model_url
    self.train_dataset = train_dataset
    self.eval_dataset = eval_dataset
    self.result_dir = result_dir
    self.tokenizer = tokenizer
    self.model = None

  def train(self, n_epochs: int, per_device_train_batch_size=6, save_total_limit=1, save_steps=500):
    if os.path.exists(f"{self.result_dir}/checkpoints") is False:
      os.mkdir(f"{self.result_dir}")
      os.mkdir(f"{self.result_dir}/checkpoints")

    last_checkpoint = glob.glob(f"{self.result_dir}/checkpoints/checkpoint")
    if len(last_checkpoint) != 0:
      model = AutoModelWithLMHead.from_pretrained(last_checkpoint[0])
    else:

      model = AutoModelWithLMHead.from_pretrained(self.model_url)

    training_args = TrainingArguments(
                    output_dir = f"{self.result_dir}/checkpoints",
                    overwrite_output_dir = True,
                    per_device_train_batch_size = per_device_train_batch_size,
                    num_train_epochs = n_epochs,
                    save_total_limit = save_total_limit,
                    save_steps=save_steps,
                    evaluation_strategy="steps"
                    )

    trainer = Trainer(
            model=model,
            args=training_args,
            data_collator=DataCollatorForLanguageModeling(tokenizer=self.tokenizer, mlm=False),
            train_dataset=self.train_dataset,
            eval_dataset=self.eval_dataset

        )
    model.to(device)
    trainer.train()
    self.model = model
    model.save_pretrained(f"{self.result_dir}/DockerRuDialoGPT-medium")


  def inference(self, text, from_locale=True,
                            top_k=10,
                            top_p=0.95,
                            num_beams=5,
                            num_return_sequences=3,
                            do_sample=True,
                            no_repeat_ngram_size=4,
                            temperature=1.3,
                            repetition_penalty=1.3,
                            length_penalty=1.0,
                            eos_token_id=50257,
                            max_new_tokens=40):

      if self.model is None:
        if from_locale:
          self.model = AutoModelWithLMHead.from_pretrained(f'{self.result_dir}')
        else:
          self.model = AutoModelWithLMHead.from_pretrained(f'{self.model_url}')
      self.model.to(device)
      inputs = self.tokenizer(text, return_tensors='pt')
      inputs.to(device)
      generated_token_ids = self.model.generate(
          **inputs,
          top_k=top_k,
          top_p=top_p,
          num_beams=num_beams,
          num_return_sequences=num_return_sequences,
          do_sample=do_sample,
          no_repeat_ngram_size=no_repeat_ngram_size,
          temperature=temperature,
          length_penalty=length_penalty,
          eos_token_id=eos_token_id,
          max_new_tokens=max_new_tokens
      )

      context_with_response = [self.tokenizer.decode(sample_token_ids) for sample_token_ids in generated_token_ids]

      return context_with_response







In [ ]:
result_dir='models'
sample = TrainPipeline(
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    result_dir=result_dir
)

In [ ]:
sample.train(1)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1479: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
500,3.949000,3.806180
1000,3.822200,3.711066
1500,3.740100,3.649034
2000,3.687700,3.606734
2500,3.663100,3.572608
3000,3.619600,3.535252
3500,3.580200,3.509503
4000,3.555200,3.485546
4500,3.526200,3.465424
5000,3.509300,3.445018


In [ ]:
model = AutoModelWithLMHead.from_pretrained("models/DockerRuDialoGPT-medium")

## Push to hub
---

In [14]:
token = ""

In [ ]:
tokenizer.push_to_hub("Vlad00k/DockerRuDialoGPT-medium", token=token, create_pr=1)
model.push_to_hub("Vlad00k/DockerRuDialoGPT-medium", token=token, create_pr=1)

## Test model
---

In [46]:
result_dir='models'
sample = TrainPipeline(
    model_url="Vlad00k/DockerRuDialoGPT-medium",
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    result_dir=result_dir
)

In [47]:
sample.inference(
  "@@ПЕРВЫЙ@@ Что такое докер образ? @@ВТОРОЙ@@", from_locale=False
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1479: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50257 for open-end generation.


['@@ПЕРВЫЙ@@ Что такое докер образ? @@ВТОРОЙ@@ это образ с докер-компоузом, который запускается при сборке контейнера  @@ПЕРВЫЙ@@@@ПЕРВЫЙ@@',
 '@@ПЕРВЫЙ@@ Что такое докер образ? @@ВТОРОЙ@@ это образ с докер-компоузом, который запускается при сборке контейнера.  @@ПЕРВЫЙ@@',
 '@@ПЕРВЫЙ@@ Что такое докер образ? @@ВТОРОЙ@@ это образ с докер-компоузом, который запускает контейнеры  @@ПЕРВЫЙ@@@@ПЕРВЫЙ@@@@ПЕРВЫЙ@@@@ПЕРВЫЙ@@@@ПЕРВЫЙ@@']

In [48]:
sample.inference(
    "@@ПЕРВЫЙ@@ Что такое контейнер docker? @@ВТОРОЙ@@", from_locale=False
)

Setting `pad_token_id` to `eos_token_id`:50257 for open-end generation.


['@@ПЕРВЫЙ@@ Что такое контейнер docker? @@ВТОРОЙ@@ контейнер с приложением и докер-компоузом  @@ПЕРВЫЙ@@@@ПЕРВЫЙ@@@@ПЕРВЫЙ@@',
 '@@ПЕРВЫЙ@@ Что такое контейнер docker? @@ВТОРОЙ@@ контейнер с приложением и докер-контейнер с приложением  @@ПЕРВЫЙ@@',
 '@@ПЕРВЫЙ@@ Что такое контейнер docker? @@ВТОРОЙ@@ это контейнер с приложением, которое запущено внутри контейнера  @@ПЕРВЫЙ@@@@ПЕРВЫЙ@@@@ПЕРВЫЙ@@']

In [49]:
sample.inference(
    "@@ПЕРВЫЙ@@ Не работает докер @@ВТОРОЙ@@", from_locale=False
)

Setting `pad_token_id` to `eos_token_id`:50257 for open-end generation.


['@@ПЕРВЫЙ@@ Не работает докер @@ВТОРОЙ@@ docker-compose.yml покажи  @@ПЕРВЫЙ@@',
 '@@ПЕРВЫЙ@@ Не работает докер @@ВТОРОЙ@@ docker-compose.yml покажи  @@ПЕРВЫЙ@@',
 '@@ПЕРВЫЙ@@ Не работает докер @@ВТОРОЙ@@ docker-compose.yml в студию  @@ПЕРВЫЙ@@']

### Как можно заметить, модель научилась отвечать по довольно узкоспециализированной теме, такой как работа с Docker.